In [ ]:
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import os
import random

# 회전 
def random_rotation(image):
    """이미지를 -40도에서 40도 사이의 임의의 각도로 회전시킵니다."""
    angle = random.uniform(-40, 40)
    return image.rotate(angle, resample=Image.BICUBIC, expand=False)

# 대칭
def random_flip(image):
    """50% 확률로 좌우 또는 상하 대칭 변환을 적용합니다."""
    if random.random() > 0.5:
        image = ImageOps.mirror(image)  # 좌우 대칭
    if random.random() > 0.5:
        image = ImageOps.flip(image)    # 상하 대칭
    return image

# 크기
def random_resize(image):
    """이미지 크기를 80%에서 120% 사이로 임의 조정합니다."""
    scale = random.uniform(0.8, 1.2)
    original_size = image.size
    new_size = (int(original_size[0] * scale), int(original_size[1] * scale))
    resized_image = image.resize(new_size, Image.BICUBIC)
    

    if scale > 1.0: 
        left = (new_size[0] - original_size[0]) / 2
        top = (new_size[1] - original_size[1]) / 2
        right = (new_size[0] + original_size[0]) / 2
        bottom = (new_size[1] + original_size[1]) / 2
        return resized_image.crop((left, top, right, bottom))
    else: 
        result = Image.new(image.mode, original_size, (0, 0, 0))
        paste_x = (original_size[0] - new_size[0]) // 2
        paste_y = (original_size[1] - new_size[1]) // 2
        result.paste(resized_image, (paste_x, paste_y))
        return result

# 어파인 변환 
def random_affine(image):
    max_translate = 10  
    max_shear = 0.1     
    
    translate_x = random.uniform(-max_translate, max_translate)
    translate_y = random.uniform(-max_translate, max_translate)
    shear_x = random.uniform(-max_shear, max_shear)
    
    return image.transform(
        image.size,
        Image.AFFINE,
        (1, shear_x, translate_x, 0, 1, translate_y),
        resample=Image.BICUBIC
    )

# 색상
def random_color_transform(image):
    """이미지의 밝기, 대비, 채도를 임의로 조절합니다."""
    #
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.8, 1.2))
    # 
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(random.uniform(0.8, 1.2))
    # 
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(random.uniform(0.8, 1.2))
    return image

def augment_image(image):
    image = random_rotation(image)
    image = random_flip(image)
    image = random_resize(image)
    image = random_affine(image)
    image = random_color_transform(image)
    return image

if __name__ == '__main__':
    input_folder = '학습이미지'
    output_folder = '학습이미지_증강'
    num_augmented_images_per_original = 5 
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.png'):
            img_path = os.path.join(input_folder, filename)
            original_image = Image.open(img_path).convert("RGB") 

            for i in range(num_augmented_images_per_original):
                augmented_img = augment_image(original_image)
                
                base_filename, file_extension = os.path.splitext(filename)
                new_filename = f"{base_filename}_aug_{i}{file_extension}"
                save_path = os.path.join(output_folder, new_filename)
                
                augmented_img.save(save_path)
    
    print(f"이미지 증강이 완료되었습니다. 결과는 '{output_folder}' 폴더에 저장되었습니다.")
